# Import Libraries

In [12]:
import pickle
import numpy as np
import pandas as pd
import csv
import json
import os
import random
import shutil
from tqdm import tqdm
from pathlib import Path
from PIL import Image
from IPython.display import display
import cv2

from keras.utils import to_categorical
from keras import layers
from keras import models

import matplotlib.pyplot as plt
from numpy.random import seed
from keras.utils import plot_model

from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras import optimizers

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img



Documentation for VGG16  
https://keras.io/applications/  
https://keras.io/applications/#vgg16  


# Import and edit images

In [13]:

image_size = 64
train_dir = os.path.join("..","dataset","trainingset")
validation_dir = os.path.join("..","dataset","testset")

print("Train Dir: ",train_dir)
print("Validation Dir: ",validation_dir)

seed(4292)

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 100
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_size, image_size),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(image_size, image_size),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Train Dir:  ..\dataset\trainingset
Validation Dir:  ..\dataset\testset
Found 432 images belonging to 4 classes.
Found 217 images belonging to 4 classes.


# Load the model

In [14]:
# create the base pre-trained model
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

# Check and freeze the model

In [15]:
# Freeze the layers except the last 4 layers
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x000002320A58A2B0> False
<keras.layers.convolutional.Conv2D object at 0x000002320A490B70> False
<keras.layers.convolutional.Conv2D object at 0x000002320A590E10> False
<keras.layers.pooling.MaxPooling2D object at 0x000002320AD0CC18> False
<keras.layers.convolutional.Conv2D object at 0x000002320AD2ABE0> False
<keras.layers.convolutional.Conv2D object at 0x000002320AD46E80> False
<keras.layers.pooling.MaxPooling2D object at 0x000002320AD9A6D8> False
<keras.layers.convolutional.Conv2D object at 0x000002320ADB8D68> False
<keras.layers.convolutional.Conv2D object at 0x000002320ADD7A90> False
<keras.layers.convolutional.Conv2D object at 0x000002320ADF56D8> False
<keras.layers.pooling.MaxPooling2D object at 0x000002320AE175C0> False
<keras.layers.convolutional.Conv2D object at 0x000002320AE31D68> False
<keras.layers.convolutional.Conv2D object at 0x000002320AE51A90> False
<keras.layers.convolutional.Conv2D object at 0x000002320AE6F6D8> False
<ker

# Create my new model

In [16]:
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_conv)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 2, 2, 512)         14714688  
_________________________________________________________________
flatten_3 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 4100      
Total params: 16,816,964
Trainable params: 9,181,700
Non-trainable params: 7,635,264
_________________________________________________________________


# Train the new model

In [17]:

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

# Train the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)
 
# Save the model
model.save('./models/my_vgg16.h5')

Epoch 1/30
1/4 [=====>........................] - ETA: 13s - loss: 1.7925 - acc: 0.2900

OSError: image file is truncated (184 bytes not processed)

# Check performance

In [48]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()


NameError: name 'history' is not defined

# See errors and images predicted

In [ ]:
# Create a generator for prediction
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(image_size, image_size),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)
 
# Get the filenames from the generator
fnames = validation_generator.filenames
 
# Get the ground truth from generator
ground_truth = validation_generator.classes
 
# Get the label to class mapping from the generator
label2index = validation_generator.class_indices
 
# Getting the mapping from class index to class label
idx2label = dict((v,k) for k,v in label2index.items())
 
# Get the predictions from the model using the generator
predictions = model.predict_generator(validation_generator, steps=validation_generator.samples/validation_generator.batch_size,verbose=1)
predicted_classes = np.argmax(predictions,axis=1)
 
errors = np.where(predicted_classes != ground_truth)[0]
print("No of errors = {}/{}".format(len(errors),validation_generator.samples))
 
# Show the errors
for i in range(len(errors)):
    pred_class = np.argmax(predictions[errors[i]])
    pred_label = idx2label[pred_class]
     
    title = 'Original label:{}, Prediction :{}, confidence : {:.3f}'.format(
        fnames[errors[i]].split('/')[0],
        pred_label,
        predictions[errors[i]][pred_class])
     
    original = load_img('{}/{}'.format(validation_dir,fnames[errors[i]]))
    plt.figure(figsize=[7,7])
    plt.axis('off')
    plt.title(title)
    plt.imshow(original)
    plt.show()